In [1]:
## IMPORTS 
import evotoon
from data_classes import CatParam, IntParam, FloatParam

import numpy as np
import pandas as pd

import random
import os

In [2]:
## MAKE SEED
SEED = evotoon.make_seed(765)
separator = "--------------------------------------------------------"

In [33]:
from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [9]:
optimal_solutions = {
	"a280" : 2579,
	"ali535" : 202339,
	"att48" : 10628,
	"att532" : 27686,
	"bayg29" : 1610,
	"bays29" : 2020,
	"berlin52" : 7542,
	"bier127" : 118282,
	"brazil58" : 25395,
	"brd14051" : 469385,
	"brg180" : 1950,
	"burma14" : 3323,
	"ch130" : 6110,
	"ch150" : 6528,
	"d198" : 15780,
	"d493" : 35002,
	"d657" : 48912,
	"d1291" : 50801,
	"d1655" : 62128,
	"d2103" : 80450,
	"d15112" : 1573084,
	"d18512" : 645238,
	"dantzig42" : 699,
	"dsj1000" : 18659688, 
	"dsj1000" : 18660188,
	"eil51" : 426,
	"eil76" : 538,
	"eil101" : 629,
	"fl417" : 11861,
	"fl1400" : 20127,
	"fl1577" : 22249,
	"fl3795" : 28772,
	"fnl4461" : 182566,
	"fri26" : 937,
	"gil262" : 2378,
	"gr17" : 2085,
	"gr21" : 2707,
	"gr24" : 1272,
	"gr48" : 5046,
	"gr96" : 55209,
	"gr120" : 6942,
	"gr137" : 69853,
	"gr202" : 40160,
	"gr229" : 134602,
	"gr431" : 171414,
	"gr666" : 294358,
	"hk48" : 11461,
	"kroA100" : 21282,
	"kroB100" : 22141,
	"kroC100" : 20749,
	"kroD100" : 21294,
	"kroE100" : 22068,
	"kroA150" : 26524,
	"kroB150" : 26130,
	"kroA200" : 29368,
	"kroB200" : 29437,
	"lin105" : 14379,
	"lin318" : 42029,
	"linhp318" : 41345,
	"nrw1379" : 56638,
	"p654" : 34643,
	"pa561" : 2763,
	"pcb442" : 50778,
	"pcb1173" : 56892,
	"pcb3038" : 137694,
	"pla7397" : 23260728,
	"pla33810" : 66048945,
	"pla85900" : 142382641,
	"pr76" : 108159,
	"pr107" : 44303,
	"pr124" : 59030,
	"pr136" : 96772,
	"pr144" : 58537,
	"pr152" : 73682,
	"pr226" : 80369,
	"pr264" : 49135,
	"pr299" : 48191,
	"pr439" : 107217,
	"pr1002" : 259045,
	"pr2392" : 378032,
	"rat99" : 1211,
	"rat195" : 2323,
	"rat575" : 6773,
	"rat783" : 8806,
	"rd100" : 7910,
	"rd400" : 15281,
	"rl1304" : 252948,
	"rl1323" : 270199,
	"rl1889" : 316536,
	"rl5915" : 565530,
	"rl5934" : 556045,
	"rl11849" : 923288,
	"si175" : 21407,
	"si535" : 48450,
	"si1032" : 92650,
	"st70" : 675,
	"swiss42" : 1273,
	"ts225" : 126643,
	"tsp225" : 3916,
	"u159" : 42080,
	"u574" : 36905,
	"u724" : 41910,
	"u1060" : 224094,
	"u1432" : 152970,
	"u1817" : 57201,
	"u2152" : 64253,
	"u2319" : 234256,
	"ulysses16" : 6859,
	"ulysses22" : 7013,
	"usa13509" : 19982859,
	"vm1084" : 239297,
	"vm1748" : 336556,
}

In [10]:
def run_acotsp(save_folder, seed, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb):
	instance_list = [
		"./ALL_tsp/a280.tsp",
		"./ALL_tsp/ali535.tsp",
		"./ALL_tsp/att48.tsp",
		"./ALL_tsp/d1655.tsp",
		"./ALL_tsp/eil101.tsp",
		"./ALL_tsp/eil51.tsp",
		"./ALL_tsp/gr202.tsp",
		"./ALL_tsp/gr229.tsp",
		"./ALL_tsp/gr431.tsp",
		"./ALL_tsp/gr666.tsp",
		"./ALL_tsp/kroA150.tsp",
		"./ALL_tsp/kroB100.tsp",
		"./ALL_tsp/kroB150.tsp",
		"./ALL_tsp/kroB200.tsp",
		"./ALL_tsp/p654.tsp",
		"./ALL_tsp/pcb1173.tsp",
		"./ALL_tsp/pr124.tsp",
		"./ALL_tsp/pr136.tsp",
		"./ALL_tsp/pr264.tsp",
		"./ALL_tsp/pr76.tsp",
		"./ALL_tsp/rat575.tsp",
		"./ALL_tsp/rat99.tsp",
		"./ALL_tsp/rd100.tsp",
		"./ALL_tsp/rl1304.tsp",
		"./ALL_tsp/rl1323.tsp",
		"./ALL_tsp/u1432.tsp",
		"./ALL_tsp/vm1748.tsp",
	]
	if not os.path.exists(save_folder):
		os.makedirs(save_folder)
	for file in instance_list:
		f_name = file.split("/")[-1].split(".")[0]
		f = open(save_folder+"/"+f_name+"-"+str(seed)+".txt", "a")
		try:
			out = evotoon.execute_ACOTSP(file, seed, optimal_solutions[f_name], "./ACOTSP-master/acotsp", alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb) * -1
		except:
			out = str((file, seed, optimal_solutions[f_name], "./ACOTSP-master/acotsp", alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb))
		f.write(str(out)+"\n")
		f.close()


In [ ]:
# EVOTOON

alpha = 2.012
beta =  7.034
rho =  0.206
ants = 25
nnls = 19
elitistants = 245
localsearch = 1
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/evotoon/1", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 2.207
beta =  6.357
rho =  0.689
ants = 87
nnls = 27
elitistants = 352
localsearch = 1
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/evotoon/2", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 2.350
beta =  4.453
rho =  0.891
ants = 78
nnls = 29
elitistants = 263
localsearch = 2
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/evotoon/3", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 8.153
beta =  9.242
rho =  0.250
ants = 66
nnls = 42
elitistants = 382
localsearch = 3
dlb = 0
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/evotoon/4", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 9.749
beta =  4.710
rho =  0.549
ants = 34
nnls = 48
elitistants = 592
localsearch = 2
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/evotoon/5", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


In [11]:
# IRACE


alpha = 3.009
beta =  8.057
rho =  0.421
ants = 73
nnls = 46
elitistants = 595
localsearch = 3
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/irace/1", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 1.264
beta =  3.501
rho =  0.357
ants = 52
nnls = 24
elitistants = 126
localsearch = 3
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/irace/2", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 1.752
beta =  4.479
rho =  0.534
ants = 63
nnls = 32
elitistants = 420
localsearch = 3
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/irace/3", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 6.118
beta =  7.286
rho =  0.316
ants = 97
nnls = 31
elitistants = 47
localsearch = 3
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/irace/4", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 2.313
beta =  8.183
rho =  0.590
ants = 22
nnls = 27
elitistants = 247
localsearch = 3
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/irace/5", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


In [ ]:
# SMAC

alpha = 4.6909356296798235
beta = 9.660321509942506
rho =  0.9307106389687687
ants = 50
nnls = 35
elitistants = 620
localsearch = 1
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/smac/1", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 7.549531688595925
beta =  3.1926625398301542
rho =  0.40282573270687994
ants = 85
nnls = 15
elitistants = 658
localsearch = 1
dlb = 0
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/smac/2", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 1.025570050576051
beta =  1.9484924435528148
rho =  0.3294451313822462
ants = 30
nnls = 33
elitistants = 379
localsearch = 3
dlb = 1
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/smac/3", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 9.680665588480524
beta =  7.459178911650364
rho =  0.8682624044554927
ants = 77
nnls = 43
elitistants = 371
localsearch = 1
dlb = 0
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/smac/4", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


alpha = 8.373444239506172
beta =  4.795027157524783
rho =  0.3777989171145002
ants = 19
nnls = 25
elitistants = 538
localsearch = 2
dlb = 0
for i in range(SEED, SEED + 5):
	run_acotsp("./acotsptesting/smac/5", i, alpha, beta, rho, ants, nnls, elitistants, localsearch, dlb)


In [28]:
def get_results_df(folder):
	cols = ["SEED " + str(i) for i in range(SEED, SEED+5)]
	dict_y = {}

	df = pd.DataFrame()
	for i in range(1,6):
		path = f"./acotsptesting/{folder}/{i}"
		for file in sorted(os.listdir(path)):
			file_path = os.path.join(path, file)
			f = open(file_path, "r")
			lines = f.readlines()
			lines = [float(line.rstrip("\n")) for line in lines]
			mean = sum(lines)/len(lines)
			row_name, seed = file.split("-")[0], file.split("-")[1].split(".")[0]
			if not dict_y.get(row_name):
				dict_y[row_name] = {}
			dict_y[row_name][seed] = mean
			f.close()

		df_y = pd.DataFrame.from_dict(dict_y, orient='index')
		df[i] = df_y.mean(axis=1)

	df.loc["Promedio"] = df.mean()
	df.loc["Mediana"] = df.median()
	df.loc["Desviación"] = df.std()
	df = df.round(3)
	
	return df


In [36]:
df1 = get_results_df("evotoon")
df2 = get_results_df("irace")
df3 = get_results_df("smac")


In [37]:
df1

,1,2,3,4,5
a280,0.000,0.000,0.000,0.527,0.442
ali535,0.000,0.000,0.000,0.000,0.000
att48,0.000,0.000,0.000,0.000,0.000
d1655,0.386,0.387,0.360,0.551,0.194
eil101,0.000,0.000,0.000,0.064,0.254
eil51,0.000,0.000,0.000,0.000,0.000
gr202,0.000,0.000,0.000,0.000,0.000
gr229,0.000,0.000,0.000,0.000,0.000
gr431,0.000,0.000,0.000,0.002,0.001
gr666,0.026,0.030,0.042,0.090,0.048


In [38]:
df2

,1,2,3,4,5
a280,0.000,0.000,0.000,0.000,0.000
ali535,0.000,0.000,0.055,0.005,0.000
att48,0.000,0.000,0.000,0.000,0.000
d1655,0.823,0.736,1.141,1.045,1.072
eil101,0.000,0.000,0.000,0.000,0.000
eil51,0.000,0.000,0.000,0.000,0.000
gr202,0.000,0.000,0.000,0.000,0.000
gr229,0.000,0.002,0.000,0.000,0.000
gr431,0.000,0.000,0.009,0.002,0.000
gr666,0.074,0.041,0.266,0.155,0.081


In [39]:
df3

,1,2,3,4,5
a280,0.000,0.016,0.000,0.000,0.000
ali535,0.000,0.002,0.025,0.000,0.000
att48,0.000,0.000,0.000,0.000,0.000
d1655,0.476,0.857,0.974,0.531,0.384
eil101,0.000,0.000,0.000,0.000,0.000
eil51,0.000,0.000,0.000,0.000,0.000
gr202,0.000,0.000,0.000,0.000,0.000
gr229,0.000,0.000,0.000,0.002,0.002
gr431,0.000,0.000,0.009,0.000,0.000
gr666,0.049,0.134,0.110,0.054,0.018


In [52]:
print("Promedio evotoon", df1.loc["Promedio"].mean())
print("Promedio irace",df2.loc["Promedio"].mean())
print("Promedio smac",df3.loc["Promedio"].mean())

Promedio evotoon 0.1442
Promedio irace 0.19080000000000003
Promedio smac 0.1258


In [53]:
df = pd.DataFrame()
df = pd.concat([df, df1.loc["Promedio"]], axis=1)
df = pd.concat([df, df2.loc["Promedio"]], axis=1)
df = pd.concat([df, df3.loc["Promedio"]], axis=1)
df.columns = ["Promedio_evotoon", "Promedio_irace", "Promedio_smac"]
print(df)
print(df.rank(axis=1))
print(df.rank(axis=1).sum())

   Promedio_evotoon  Promedio_irace  Promedio_smac
1             0.092           0.186          0.096
2             0.094           0.113          0.183
3             0.095           0.265          0.184
4             0.209           0.201          0.098
5             0.231           0.189          0.068
   Promedio_evotoon  Promedio_irace  Promedio_smac
1               1.0             3.0            2.0
2               1.0             2.0            3.0
3               1.0             3.0            2.0
4               3.0             2.0            1.0
5               3.0             2.0            1.0
Promedio_evotoon     9.0
Promedio_irace      12.0
Promedio_smac        9.0
dtype: float64
